# Phising Email Sentiment Analysis

# Imports

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import AutoTokenizer

from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

from tensorflow.keras.layers import Dense, Bidirectional, Embedding, LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Functions

In [8]:
def check_for_nulls_dupes(df):
  print(f"Amount of Null Values: {df.isna().sum()}")
  print(f"Amount of Dupe Values: {df.duplicated().sum()}")

# EDA Analysis

In [7]:
df = pd.read_csv("/Users/maukanmir/Downloads/archive/phishing_email.csv")
df

,text_combined,label
0,hpl nom may 25 2001 see attached file hplno 52...,0
1,nom actual vols 24 th forwarded sabrae zajac h...,0
2,enron actuals march 30 april 1 201 estimated a...,0
3,hpl nom may 30 2001 see attached file hplno 53...,0
4,hpl nom june 1 2001 see attached file hplno 60...,0
...,...,...
82481,info advantageapartmentscom infoadvantageapart...,1
82482,monkeyorg helpdeskmonkeyorg monkeyorg hi josep...,1
82483,help center infohelpcentercoza_infohelpcenterc...,1
82484,metamask infosofamekarcom verify metamask wall...,1


# Check for Nulls and Dupes

In [9]:
check_for_nulls_dupes(df)

Amount of Null Values: text_combined    0
label            0
dtype: int64
Amount of Dupe Values: 408


In [10]:
df = df.drop_duplicates()